### Import libraries

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
from skimage import io
import sys
import seaborn as sns
from scipy import spatial

In [2]:
sys.path.insert(1, "/Users/k1801626/OneDrive - King's College London/git/AFT-Alignment_by_Fourier_Transform/Python_implementation/")
import AFT_tools as AFT       

In [88]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

### Import data (csv tracks and images)

In [3]:
csv_folder = "/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/"
image_folder = "/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/images/"

In [5]:
spots_list = glob.glob(csv_folder+'*Spots.csv')

print(*spots_list, sep="\n")

/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/merged_Spots.csv


In [6]:
tracks_list = glob.glob(csv_folder+'*Tracks.csv')

print(*tracks_list, sep="\n")

/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/merged_Tracks.csv


In [10]:
image_list = glob.glob(image_folder+'*.tif')

print(*image_list, sep="\n")

/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/images/Position_6_AFT.tif
/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/images/Position_12_AFT.tif


### Parameters

In [104]:
output_image_folder = csv_folder+'output_images/'

In [38]:
single_frame = False

In [12]:
#### required AFT parameters ####
window_size = 100
overlap = 0.6
neighborhood_radius = 5

### Load data

In [14]:
df_spots = pd.read_csv(spots_list[0])
df_spots.head()

,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,...,PERIMETER,CIRCULARITY,SOLIDITY,SHAPE_INDEX,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID
0,ID2560,2560,1,113788.0,446.481612,463.926373,0.0,26.0,26,190.336445,...,1850.608931,0.417613,0.884730,5.485520,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
1,ID2435,2435,1,64912.0,186.902997,251.383041,0.0,1.0,1,143.727227,...,1103.639245,0.669551,0.961309,4.332246,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
2,ID2563,2563,1,111268.0,432.193334,454.720643,0.0,27.0,27,188.187463,...,1845.354851,0.410564,0.845194,5.532409,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
3,ID2437,2437,1,71143.0,230.466989,274.844695,0.0,4.0,4,150.484286,...,1161.466995,0.662717,0.959538,4.354525,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
4,ID2565,2565,1,123560.0,441.649200,460.855612,0.0,28.0,28,198.340124,...,1885.321437,0.436928,0.854008,5.362902,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1


In [15]:
df_tracks = pd.read_csv(tracks_list[0])
df_tracks.head()

,LABEL,TRACK_INDEX,TRACK_ID,DIVISION_TIME_MEAN,DIVISION_TIME_STD,NUMBER_SPOTS,NUMBER_GAPS,NUMBER_SPLITS,NUMBER_MERGES,NUMBER_COMPLEX,...,CONFINEMENT_RATIO,MEAN_STRAIGHT_LINE_SPEED,LINEARITY_OF_FORWARD_PROGRESSION,MEAN_DIRECTIONAL_CHANGE_RATE,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID
0,Track_1,1,1,NaN,NaN,76,0,0,0,0,...,0.651187,10.556033,0.651187,1.235458,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
1,Track_2,2,2,NaN,NaN,55,0,0,0,0,...,0.013550,0.361223,0.013550,1.681933,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_2
2,Track_3,3,3,NaN,NaN,76,0,0,0,0,...,0.149645,1.959934,0.149645,1.803235,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_3
3,Track_4,4,4,NaN,NaN,76,0,0,0,0,...,0.044778,0.713082,0.044778,1.874086,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_4
4,Track_8,8,8,NaN,NaN,29,0,0,0,0,...,0.140809,2.646769,0.140809,1.866365,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_8


In [16]:
movie_list = df_spots.File_name_raw.unique()
print(*movie_list, sep="\n")

Position_26
Position_27
Position_31
Position_33
Position_6
Position_12


### Find images for current experimental condition

In [35]:
im_list_current = []
for im_file in range(len(image_list)):
    temp_im_file = image_list[im_file].split("/")[-1].split("_",2)[:2]
    temp_im_file = '_'.join(temp_im_file)
    im_list_current = np.append(im_list_current, temp_im_file)

print(im_list_current)

['Position_6' 'Position_12']


### Run analysis

In [81]:
df_subset_out = pd.DataFrame()
for position in range(len(im_list_current)):

    # load current image
    im = io.imread(image_list[position])
    if single_frame == True:
        im = im[0,]

    # run AFT on all (relevant) frames
    x, y, u, v, im_theta, im_eccentricity = AFT.image_local_order(im, window_size, overlap, save_path = [],
                                                                  plot_overlay=False, 
                                                                  plot_angles=False,
                                                                  plot_eccentricity=False,
                                                                  save_figures=False)
    # get coords for AFT grid
    AFT_coords = np.empty(shape=(len(x),2))
    AFT_coords[:,0] = x
    AFT_coords[:,1] = y

    # get relevant part of spots csv
    df_subset = df_spots.loc[df_spots.File_name_raw == im_list_current[position]]

    # initialise df for ouput of track analysis
    df_track_out = pd.DataFrame(columns = ['TRACK_ID','FRAME','track_angle','AFT_angle','AFT_track_angle','AFT_track_angle_cos2',
                                    'track_angle_u', 'track_angle_v','AFT_angle_u','AFT_angle_v'])

    # for each track
    for trackID in range(len(df_subset.TRACK_ID.unique())):
    
        # create df for current track
        df_track = df_subset.loc[df_subset.TRACK_ID == df_subset.TRACK_ID.unique()[trackID]]
        df_track = df_track.sort_values(by='FRAME')
        df_track = df_track.reset_index(drop=True)
        df_track = df_track.drop_duplicates(subset=['FRAME'], ignore_index=True)
    
        # initialise output arrays
        AFT_angle = []
        track_angle = []
        current_time_point = []
    
        # calculate angles for AFT and track
        if single_frame == False:
            for time_point in range(len(df_track)-1):
                
                track_current = [df_track.loc[time_point, 'POSITION_X'], df_track.loc[time_point, 'POSITION_Y']]
                track_next = [df_track.loc[time_point+1, 'POSITION_X'], df_track.loc[time_point+1, 'POSITION_Y']]
                
                d_closest,idx_closest = spatial.KDTree(AFT_coords).query(track_current)
                AFT_angle = np.append(AFT_angle, np.ravel(im_theta[time_point])[idx_closest])
                
                track_length = [track_next[0]-track_current[0], track_next[1]-track_current[1]]
                track_norm = np.sqrt(track_length[0] ** 2 + track_length[1] ** 2)
                track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
                track_angle = np.append(track_angle, np.arctan2(track_direction[1], track_direction[0]))
            
                current_time_point = np.append(current_time_point, time_point)
        else:
            for time_point in range(len(df_track)-1):
                
                track_current = [df_track.loc[time_point, 'POSITION_X'], df_track.loc[time_point, 'POSITION_Y']]
                track_next = [df_track.loc[time_point+1, 'POSITION_X'], df_track.loc[time_point+1, 'POSITION_Y']]
                
                d_closest,idx_closest = spatial.KDTree(AFT_coords).query(track_current)
                AFT_angle = np.append(AFT_angle, np.ravel(im_theta)[idx_closest])
                
                track_length = [track_next[0]-track_current[0], track_next[1]-track_current[1]]
                track_norm = np.sqrt(track_length[0] ** 2 + track_length[1] ** 2)
                track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
                track_angle = np.append(track_angle, np.arctan2(track_direction[1], track_direction[0]))
            
                current_time_point = np.append(current_time_point, time_point)
    
        # calculate angle difference and cosine squared of the angle difference
        AFT_track_angle = AFT_angle-track_angle
        AFT_track_angle_cos2 = np.cos(AFT_track_angle) ** 2

        # create temporary df
        df_out = pd.DataFrame(columns = ['TRACK_ID','FRAME','track_angle','AFT_angle','AFT_track_angle','AFT_track_angle_cos2',
                                    'track_angle_u', 'track_angle_v','AFT_angle_u','AFT_angle_v'])
    
        df_out.TRACK_ID = np.full((len(df_track)-1, ), df_subset.TRACK_ID.unique()[trackID])
        df_out.FRAME = current_time_point
        df_out.track_angle = track_angle
        df_out.AFT_angle = AFT_angle
        df_out.AFT_track_angle = AFT_track_angle
        df_out.AFT_track_angle_cos2 = AFT_track_angle_cos2
        
        df_out.track_angle_u = np.cos(track_angle)
        df_out.track_angle_v = np.sin(track_angle)
        df_out.AFT_angle_u = np.cos(AFT_angle)
        df_out.AFT_angle_v = np.sin(AFT_angle)

        df_out_merge = pd.merge(df_track, df_out, on=['TRACK_ID','FRAME'], how='outer')

        # concatenate output df
        df_list = [df_track_out, df_out_merge]
        df_track_out = pd.concat([df_track_out for df_track_out in df_list if not df_track_out.empty])

    df_list1 = [df_subset_out, df_track_out]
    df_subset_out = pd.concat([df_subset_out for df_subset_out in df_list1 if not df_subset_out.empty])

/var/folders/b8/814qyc5s5jx004x1r459c2gc0000gs/T/ipykernel_72169/169628012.py:53: RuntimeWarning: invalid value encountered in scalar divide
  track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
/var/folders/b8/814qyc5s5jx004x1r459c2gc0000gs/T/ipykernel_72169/169628012.py:53: RuntimeWarning: invalid value encountered in scalar divide
  track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
/var/folders/b8/814qyc5s5jx004x1r459c2gc0000gs/T/ipykernel_72169/169628012.py:53: RuntimeWarning: invalid value encountered in scalar divide
  track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]


In [145]:
df_subset_out.to_excel(image_folder+'AFT_tracks_df_out.xlsx')

### Plot

In [142]:
cmap = plt.get_cmap('hsv')
for position in range(len(im_list_current)):
    # load current image
    im = io.imread(image_list[position])
    if single_frame == True:
        im = im[0,]
    
    # run AFT on all (relevant) frames
    x, y, u, v, im_theta, im_eccentricity = AFT.image_local_order(im, window_size, overlap,
                                                                  intensity_thresh =150, save_path = [],
                                                                  plot_overlay=False, 
                                                                  plot_angles=False,
                                                                  plot_eccentricity=False,
                                                                  save_figures=False)
    # get relevant part of subset_out df
    df_temp = df_subset_out.loc[df_subset_out.File_name_raw == im_list_current[position]]

    # for each timepoint
    for time_point in range(len(df_temp.FRAME.unique())-1):
        plt.imshow(im[time_point,], cmap='gray')
        for track in range(len(df_temp.TRACK_ID.unique())):
            plt.plot(df_temp.loc[df_temp.TRACK_ID == df_temp.TRACK_ID.unique()[track]]['POSITION_X'], df_temp.loc[df_temp.TRACK_ID == df_temp.TRACK_ID.unique()[track]]['POSITION_Y'], linewidth=1, color='w')
    
        plt.quiver(x,y,u[time_point],v[time_point], color='yellow', pivot='mid', scale_units='xy', 
                   scale=overlap, headaxislength=0, headlength=0, width=0.005, alpha=0.4)
        plt.plot(df_temp.loc[time_point,'POSITION_X'], df_temp.loc[time_point,'POSITION_Y'], 
                 marker='o', linestyle='None', color=cmap(time_point*2))
        plt.axis('off')
        plt.savefig(output_image_folder+str(df_temp.File_name_raw.unique()[0])+'_AFT_tracks_'+str(time_point)+'.png', dpi=300, bbox_inches='tight')
        plt.clf()  

<Figure size 640x480 with 0 Axes>